# Final Project: Euchre
Tyler Gibson & Carter Shavitz | CSC 5661

https://en.wikipedia.org/wiki/Euchre

https://cardgames.io/euchre/

In [1]:
import random
from player import Player
from score import Score

import numpy as np
import re
import time

import tqdm

## Euchre Environment

In [ ]:
class Euchre:
    """
    A class to manage the euchre board.
    """

    def __init__(self, config: dict) -> None:
        """
        The constructor for the class.

        Parameters
        ----------
        config : dict
            The configuration parameters for euchre.
        """
        self.config = config
        self.score_to_win = self.config['score_to_win']  # 5, 7, or 10
        self.values = self.config['values']  # 9 - A that can include 7's and/or 8's

        self.num_players = 4
        self.suits = ['♠', '♥', '♦', '♣']
        self.deck = [f"{value}{suit}" for suit in self.suits for value in self.values]
        self.card_state = {f"{value}{suit}": False for suit in self.suits for value in self.values}
        self.trick_state = {f"{value}{suit}": False for suit in self.suits for value in self.values}


        self.score = Score()

        self.team_1_score = 0
        self.team_2_score = 0
        
        self.top_card = None
        self.trump = None
        self.dealer = 1
        self.lead_suit = ''

    def deal_cards(self, distribution_pattern=(3, 2)):
        """
        Deal the cards for a new 

        Parameters
        ----------
        

        Returns
        -------

        """
        random.shuffle(self.deck)
    
        players_hands = {
            '1': [],
            '2': [],
            '3': [],
            '4': []
        }

        deck_index = 0
        for count, player in enumerate(list(players_hands.keys()) * 2):
            if (count % 2 == 0 and count < len(players_hands)) or (count % 2 == 1 and count >= len(players_hands)):
                players_hands[player].append(self.deck[deck_index:deck_index+distribution_pattern[0]])
                deck_index += distribution_pattern[0] 
            else:
                players_hands[player].append(self.deck[deck_index:deck_index+distribution_pattern[1]])
                deck_index += distribution_pattern[1]

        for key in players_hands.keys():
            players_hands[key] =  [item for sublist in players_hands[key] for item in sublist]
                    
        return players_hands, self.deck[deck_index]
    
    def reset_game(self, players: list[Player]):
        """
        Reset the game.
        """
        for player in players:
            player.team_id = 2 if player.team_id == 1 else 1

        self.team_1_score = 0
        self.team_2_score = 0

        self.top_card = None
        self.trump = None

        self.dealer = np.random.choice(range(self.num_players)) + 1

        players = self.reset_hand(players=players)

        return players

    def reset_hand(self, players: list[Player]):
        """
        Reset the hand.
        """
        self.card_state = {f"{value}{suit}": False for suit in self.suits for value in self.values}

        for player in players:
            player.points = 0
            
        players_hands, top_card = self.deal_cards(distribution_pattern=(3, 2))

        for count, player in enumerate(players):
            player.set_hand(players_hands[str(count + 1)])

        self.top_card = top_card

        self.reset_trick()
        
        return players

    def reset_trick(self):
        """
        Reset the trick.
        """
        self.dealer = 1 if self.dealer == 4 else self.dealer + 1  
        self.trick_state = {f"{value}{suit}": False for suit in self.suits for value in self.values}


    def reward(self, won_trick, teammate_won) -> float:
        """
        TODO

        Returns
        -------
        float
            The reward based on the action.
        """
        return 1

    def step(self, players: list[Player]) -> dict:
        """
        TODO

        Parameters
        ----------
        a : TODO
            TODO

        Returns
        -------
        dict
            TODO
        """
        trump_suit =  re.findall(self.score.suits, self.top_card)[0]  # determine the trump suit

        # find the player id who won the trick
        highest_player_id = self.score.score_trick(
            players=players,
            trump_suit=trump_suit,
            lead_suit=self.lead_suit
        )
        highest_player = [player for player in players if player.id == highest_player_id][0]  # get player with won trick
        highest_player.points += 1  # update their points by 1 for the hand
        current_player_id = highest_player_id  # the player who won the trick will lead the next trick

        for x in range(self.num_players):
            current_player = [player for player in players if player.id == current_player_id][0]  # get current player based on id

        return {
            'state': self.card_state,
            'rewards': 
        }
        

In [90]:
env_config = {
    'score_to_win': 10,
    'values': ['9','10','J','Q','K','A'],
}

euchre = Euchre(env_config)

score = Score()

In [91]:
players = [Player(id=id + 1, team_id=1 if id % 2 == 0 else 2) for id in range(euchre.num_players)]

euchre.reset_hand(players=players)

for player in players:
    print(player.hand, player.id, player.team_id)

euchre.top_card, euchre.dealer

TypeError: __init__() missing 1 required positional argument: 'agent_config'

In [92]:
agent_config = {
    'gamma': 0.9,  # the discount factor
    'epsilon': 0.1,  # the epsilon-greedy parameter
    'alpha': 0.001,  # the learning rate
    'input_size': 25,
    'hidden_size': 128,  # the hidden layer size
    'A': euchre.deck,  # the action set
    'M': 100000, # set the memory size
    'B': 64,  # set the batch size
    'C': 500,  # when to update the target approximator
    'n_steps': 100,  # the number of steps to use to update
    'epsilon_burnin': 10000,
    'double_dqn': True,
    'prioritized_replay': True,
    'multi_step': True,
    'omega': 0.05,
    'beta': 0.5,
    'n_step_return': 20,
} 

players = [Player(id=id + 1, team_id=1 if id % 2 == 0 else 2, agent_config=agent_config) for id in range(euchre.num_players)]

In [93]:
players = euchre.reset_game(players=players)

hands = 1  # keep track of which game we are on
while euchre.team_1_score < euchre.score_to_win and euchre.team_2_score < euchre.score_to_win:  # while game not complete
    current_player_id = 1 if euchre.dealer == 4 else euchre.dealer + 1  # get the player to the left of the dealer to start the hand
    player_choose_trump = [player for player in players if player.id == current_player_id][0]  # assuming the player to the left of dealer chooses trump and starts (not based off of actual rules)
    player_choose_trump.trick_team = 'makers'  # assign the first player as the Makers

    makers = player_choose_trump.team_id
    for player in players:
        player.trick_team = 'makers' if player.team_id == makers else 'defenders'

    # loop through 5 tricks (each player should have 5 cards)
    for _ in range(5):
        lead_suit = ''  # init lead suit to empty

        # loop through each player for their turn
        for x in range(euchre.num_players):
            current_player = [player for player in players if player.id == current_player_id][0]  # get current player based on id
            current_player.current_card = current_player.choose_card(random=True)  # set the current card of the player

            # # update card state based on current card chosen
            # euchre.card_state[(
            #     re.split(euchre.score.suits, current_player.current_card)[0],
            #     re.findall(euchre.score.suits, current_player.current_card)[0]
            # )] = True

            # if it is the first player, determine the lead suit
            if x == 0:
                lead_suit = re.findall(score.suits, current_player.current_card)[0] # use regex

            current_player_id = 1 if current_player_id == 4 else current_player_id + 1  # update the current player id to the next player

        trump_suit =  re.findall(score.suits, euchre.top_card)[0]  # determine the trump suit

        # find the player id who won the trick
        highest_player_id = score.score_trick(
            players=players,
            trump_suit=trump_suit,
            lead_suit=lead_suit
        )
        highest_player = [player for player in players if player.id == highest_player_id][0]  # get player with won trick
        highest_player.points += 1  # update their points by 1 for the hand
        current_player_id = highest_player_id  # the player who won the trick will lead the next trick

    hand_scores = score.score_hand(players, solo_call=False)  # calculate the hand scores by looking at each players points for all tricks
    
    # update team scores
    if player_choose_trump.team_id == 1:
        euchre.team_1_score += hand_scores['makers']
        euchre.team_2_score += hand_scores['defenders']
    else:
        euchre.team_2_score += hand_scores['makers']
        euchre.team_1_score += hand_scores['defenders']

    print(f'Hand {hands}\t\t Team 1 Score: {euchre.team_1_score}\t\tTeam 2 Score: {euchre.team_2_score}')
    hands += 1  # update hand counter
    players = euchre.reset_hand(players=players)  # reset hand

print('Team 1 Wins' if euchre.team_1_score >= 10 else 'Team 2 Wins')

Hand 1		 Team 1 Score: 2		Team 2 Score: 0
Hand 2		 Team 1 Score: 3		Team 2 Score: 0
Hand 3		 Team 1 Score: 5		Team 2 Score: 0
Hand 4		 Team 1 Score: 5		Team 2 Score: 2
Hand 5		 Team 1 Score: 5		Team 2 Score: 3
Hand 6		 Team 1 Score: 5		Team 2 Score: 5
Hand 7		 Team 1 Score: 5		Team 2 Score: 6
Hand 8		 Team 1 Score: 5		Team 2 Score: 8
Hand 9		 Team 1 Score: 5		Team 2 Score: 9
Hand 10		 Team 1 Score: 7		Team 2 Score: 9
Hand 11		 Team 1 Score: 7		Team 2 Score: 10
Team 2 Wins


## Agent Creation

In [94]:
agent_config = {
    'gamma': 0.9,  # the discount factor
    'epsilon': 0.1,  # the epsilon-greedy parameter
    'alpha': 0.001,  # the learning rate
    'input_size': 25,
    'hidden_size': 128,  # the hidden layer size
    'A': euchre.deck,  # the action set
    'M': 100000, # set the memory size
    'B': 64,  # set the batch size
    'C': 500,  # when to update the target approximator
    'n_steps': 100,  # the number of steps to use to update
    'epsilon_burnin': 10000,
    'double_dqn': True,
    'prioritized_replay': True,
    'multi_step': True,
    'omega': 0.05,
    'beta': 0.5,
    'n_step_return': 20,
} 

players = [Player(id=id + 1, team_id=1 if id % 2 == 0 else 2, agent_config=agent_config) for id in range(euchre.num_players)]

In [ ]:
count = 0

for epi in tqdm.tqdm(range(25000)):
    players = euchre.reset_game(players=players)
    hands = 1  # keep track of which game we are on
    while euchre.team_1_score < euchre.score_to_win and euchre.team_2_score < euchre.score_to_win:  # while game not complete
        count += 1
        
        current_player_id = 1 if euchre.dealer == 4 else euchre.dealer + 1  # get the player to the left of the dealer to start the hand
        player_choose_trump = [player for player in players if player.id == current_player_id][0]  # assuming the player to the left of dealer chooses trump and starts (not based off of actual rules)
        player_choose_trump.trick_team = 'makers'  # assign the first player as the Makers

        makers = player_choose_trump.team_id
        for player in players:
            player.trick_team = 'makers' if player.team_id == makers else 'defenders'

        # loop through 5 tricks (each player should have 5 cards)
        for _ in range(5):
            euchre.lead_suit = ''  # init lead suit to empty

            round_state = euchre.card_state.copy()
            trick_state = euchre.trick_state.copy()
            actions = {
                
            }
            # loop through each player for their turn
            for x in range(euchre.num_players):
                data_t = {}    #init the data for this turn
                data_t['s_t'] = np.array(
                    [1 if round_state[card] else 0 for card in euchre.deck] + 
                    [1 if trick_state[card] else 0 for card in euchre.deck]
                ).flatten().tolist()

                current_player = [player for player in players if player.id == current_player_id][0]  # get current player based on id
                current_player.current_card = current_player.agent.pi(s_t=data_t['s_t'], epsilon=current_player.agent.epsilon_t(count, epi))  # set the current card of the player
                data_t['a_t'] = [current_player.current_card]    #record the action at t
                data_t['d_s_a'] = data_t['s_t'] + data_t['a_t']    #make the state action pair

                # if it is the first player, determine the lead suit
                if x == 0:
                    lead_suit = re.findall(score.suits, current_player.current_card)[0] # use regex

                current_player_id = 1 if current_player_id == 4 else current_player_id + 1  # update the current player id to the next player

                current_player.data_t = data_t

                # update card state based on current card chosen
                euchre.card_state[current_player.current_card] = True
                euchre.trick_state[current_player.current_card] = True

            euchre.step(players)

            # trump_suit =  re.findall(score.suits, euchre.top_card)[0]  # determine the trump suit

            # # find the player id who won the trick
            # highest_player_id = score.score_trick(
            #     players=players,
            #     trump_suit=trump_suit,
            #     lead_suit=lead_suit
            # )
            # highest_player = [player for player in players if player.id == highest_player_id][0]  # get player with won trick
            # highest_player.points += 1  # update their points by 1 for the hand
            # current_player_id = highest_player_id  # the player who won the trick will lead the next trick

            # for x in range(euchre.num_players):
            #     current_player = [player for player in players if player.id == current_player_id][0]  # get current player based on id
            #     euchre.step(current_player.current_card)

            euchre.reset_trick()

        hand_scores = score.score_hand(players, solo_call=False)  # calculate the hand scores by looking at each players points for all tricks
        
        # update team scores
        if player_choose_trump.team_id == 1:
            euchre.team_1_score += hand_scores['makers']
            euchre.team_2_score += hand_scores['defenders']
        else:
            euchre.team_2_score += hand_scores['makers']
            euchre.team_1_score += hand_scores['defenders']

        print(f'Hand {hands}\t\t Team 1 Score: {euchre.team_1_score}\t\tTeam 2 Score: {euchre.team_2_score}')
        hands += 1  # update hand counter
        players = euchre.reset_hand(players=players)  # reset hand

    print('Team 1 Wins' if euchre.team_1_score >= 10 else 'Team 2 Wins')

  0%|          | 68/25000 [00:00<01:13, 340.28it/s]

Hand 1		 Team 1 Score: 2		Team 2 Score: 0
Hand 2		 Team 1 Score: 4		Team 2 Score: 0
Hand 3		 Team 1 Score: 6		Team 2 Score: 0
Hand 4		 Team 1 Score: 8		Team 2 Score: 0
Hand 5		 Team 1 Score: 10		Team 2 Score: 0
Team 1 Wins
Hand 1		 Team 1 Score: 2		Team 2 Score: 0
Hand 2		 Team 1 Score: 4		Team 2 Score: 0
Hand 3		 Team 1 Score: 6		Team 2 Score: 0
Hand 4		 Team 1 Score: 8		Team 2 Score: 0
Hand 5		 Team 1 Score: 10		Team 2 Score: 0
Team 1 Wins
Hand 1		 Team 1 Score: 0		Team 2 Score: 2
Hand 2		 Team 1 Score: 0		Team 2 Score: 4
Hand 3		 Team 1 Score: 0		Team 2 Score: 6
Hand 4		 Team 1 Score: 0		Team 2 Score: 8
Hand 5		 Team 1 Score: 0		Team 2 Score: 10
Team 2 Wins
Hand 1		 Team 1 Score: 2		Team 2 Score: 0
Hand 2		 Team 1 Score: 4		Team 2 Score: 0
Hand 3		 Team 1 Score: 6		Team 2 Score: 0
Hand 4		 Team 1 Score: 8		Team 2 Score: 0
Hand 5		 Team 1 Score: 10		Team 2 Score: 0
Team 1 Wins
Hand 1		 Team 1 Score: 0		Team 2 Score: 2
Hand 2		 Team 1 Score: 0		Team 2 Score: 4
Hand 3		 Team 1 Score: 0

  1%|          | 174/25000 [00:00<01:12, 342.14it/s]

Hand 2		 Team 1 Score: 0		Team 2 Score: 4
Hand 3		 Team 1 Score: 0		Team 2 Score: 6
Hand 4		 Team 1 Score: 0		Team 2 Score: 8
Hand 5		 Team 1 Score: 0		Team 2 Score: 10
Team 2 Wins
Hand 1		 Team 1 Score: 2		Team 2 Score: 0
Hand 2		 Team 1 Score: 4		Team 2 Score: 0
Hand 3		 Team 1 Score: 6		Team 2 Score: 0
Hand 4		 Team 1 Score: 8		Team 2 Score: 0
Hand 5		 Team 1 Score: 10		Team 2 Score: 0
Team 1 Wins
Hand 1		 Team 1 Score: 2		Team 2 Score: 0
Hand 2		 Team 1 Score: 4		Team 2 Score: 0
Hand 3		 Team 1 Score: 6		Team 2 Score: 0
Hand 4		 Team 1 Score: 8		Team 2 Score: 0
Hand 5		 Team 1 Score: 10		Team 2 Score: 0
Team 1 Wins
Hand 1		 Team 1 Score: 2		Team 2 Score: 0
Hand 2		 Team 1 Score: 4		Team 2 Score: 0
Hand 3		 Team 1 Score: 6		Team 2 Score: 0
Hand 4		 Team 1 Score: 8		Team 2 Score: 0
Hand 5		 Team 1 Score: 10		Team 2 Score: 0
Team 1 Wins
Hand 1		 Team 1 Score: 2		Team 2 Score: 0
Hand 2		 Team 1 Score: 4		Team 2 Score: 0
Hand 3		 Team 1 Score: 6		Team 2 Score: 0
Hand 4		 Team 1 Score: 8

  1%|          | 245/25000 [00:00<01:11, 345.09it/s]

...

  4%|▎         | 884/25000 [00:02<01:11, 337.12it/s]

Hand 1		 Team 1 Score: 2		Team 2 Score: 0
Hand 2		 Team 1 Score: 4		Team 2 Score: 0
Hand 3		 Team 1 Score: 6		Team 2 Score: 0
Hand 4		 Team 1 Score: 8		Team 2 Score: 0
Hand 5		 Team 1 Score: 10		Team 2 Score: 0
Team 1 Wins
Hand 1		 Team 1 Score: 2		Team 2 Score: 0
Hand 2		 Team 1 Score: 4		Team 2 Score: 0
Hand 3		 Team 1 Score: 6		Team 2 Score: 0
Hand 4		 Team 1 Score: 8		Team 2 Score: 0
Hand 5		 Team 1 Score: 10		Team 2 Score: 0
Team 1 Wins
Hand 1		 Team 1 Score: 0		Team 2 Score: 2
Hand 2		 Team 1 Score: 0		Team 2 Score: 4
Hand 3		 Team 1 Score: 0		Team 2 Score: 6
Hand 4		 Team 1 Score: 0		Team 2 Score: 8
Hand 5		 Team 1 Score: 0		Team 2 Score: 10
Team 2 Wins
Hand 1		 Team 1 Score: 0		Team 2 Score: 2
Hand 2		 Team 1 Score: 0		Team 2 Score: 4
Hand 3		 Team 1 Score: 0		Team 2 Score: 6
Hand 4		 Team 1 Score: 0		Team 2 Score: 8
Hand 5		 Team 1 Score: 0		Team 2 Score: 10
Team 2 Wins
Hand 1		 Team 1 Score: 0		Team 2 Score: 2
Hand 2		 Team 1 Score: 0		Team 2 Score: 4
Hand 3		 Team 1 Score: 0

  4%|▎         | 931/25000 [00:02<01:15, 317.65it/s]


Hand 1		 Team 1 Score: 2		Team 2 Score: 0
Hand 2		 Team 1 Score: 4		Team 2 Score: 0
Hand 3		 Team 1 Score: 6		Team 2 Score: 0
Hand 4		 Team 1 Score: 8		Team 2 Score: 0
Hand 5		 Team 1 Score: 10		Team 2 Score: 0
Team 1 Wins
Hand 1		 Team 1 Score: 2		Team 2 Score: 0
Hand 2		 Team 1 Score: 4		Team 2 Score: 0
Hand 3		 Team 1 Score: 6		Team 2 Score: 0
Hand 4		 Team 1 Score: 8		Team 2 Score: 0
Hand 5		 Team 1 Score: 10		Team 2 Score: 0
Team 1 Wins
Hand 1		 Team 1 Score: 2		Team 2 Score: 0
Hand 2		 Team 1 Score: 4		Team 2 Score: 0
Hand 3		 Team 1 Score: 6		Team 2 Score: 0
Hand 4		 Team 1 Score: 8		Team 2 Score: 0
Hand 5		 Team 1 Score: 10		Team 2 Score: 0
Team 1 Wins
Hand 1		 Team 1 Score: 2		Team 2 Score: 0
Hand 2		 Team 1 Score: 4		Team 2 Score: 0
Hand 3		 Team 1 Score: 6		Team 2 Score: 0
Hand 4		 Team 1 Score: 8		Team 2 Score: 0
Hand 5		 Team 1 Score: 10		Team 2 Score: 0
Team 1 Wins
Hand 1		 Team 1 Score: 2		Team 2 Score: 0
Hand 2		 Team 1 Score: 4		Team 2 Score: 0
Hand 3		 Team 1 Score: 6

KeyboardInterrupt: 